

### Course Number: INFO 7374
### Assignment: 4


---


### *Student Name: Krishnakanth Naik Jarapala*
### *NU-ID: 002724795*



# Direct Preference Optimiation - Finetuning

In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 12.1 MB/s eta 0:00:0000:0100:01


In [2]:
!pip install huggingface_hub trl sentencepiece transformers peft datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 5.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: docstring-parser
    Found existing installation: docstring-parser 0.15
    Uninstalling docstring-parser-0.15:
      Successfully uninstalled docstring-parser-0.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.


In [3]:
!pip install --upgrade trl

#### Import required Packages

In [4]:
# Importing packages
import os
import gc
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
from trl import DPOTrainer, DPOConfig
import bitsandbytes as bnb
from tqdm import tqdm

2024-08-06 05:12:20.807056: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 05:12:20.807163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 05:12:20.940679: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Import the notebook_login method
from huggingface_hub import notebook_login

# Log in interactively
notebook_login()

In [6]:
from datasets import load_dataset
dataset = load_dataset("jkkn/Mistral-7B-Instruct-v2.0-Lima-PairRM-DPO-Dataset")['train']

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

In [7]:
def extract_assistant_answer(example):
    # Find the assistant's answer in the list of messages
    chosen_answer = ""
    for message in example['chosen']:
        if message['role'] == 'assistant':
            chosen_answer = message['content']
            break

    rejected_answer = ""
    for message in example['rejected']:
        if message['role'] == 'assistant':
            rejected_answer = message['content']
            break

    return {
            "prompt": example['prompt'],
            "chosen": chosen_answer,
            'rejected': rejected_answer,
            }

# Save columns
cols_to_drop = dataset.column_names


# Format dataset
dataset_modified = dataset.map(
    extract_assistant_answer,
    remove_columns=cols_to_drop
)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [8]:
dataset_modified[1]

{'prompt': 'This claim was made popular by being said in the movie The Social Network. It exactly says:\n\n> Did you know there are more people with genius IQs living in China than there are people of any kind living in the United States?\n',
 'chosen': 'Yes, that is a statement made in the movie "The Social Network," specifically in a scene where a character is discussing the potential reach of a social media platform. However, it\'s important to note that the claim itself is a matter of debate among researchers. While it\'s true that there are more people with higher IQs in China than there are people in the United States, according to some studies, it\'s not clear that the absolute number of people with genius-level IQs is higher in China than in the US.\n\nFurthermore, IQ tests may not be the best measure of intelligence and intelligence itself is a complex construct that cannot be accurately captured by a single number. It\'s also worth noting that the concept of "genius IQ" is no

#### Load Model and their Configuarations

In [9]:
# Define model names and tokens
peft_model_name = "mistralai/Mistral-7B-Instruct-v0.2" # The model obtained after the SFT step
new_model_name = "Mistral-7B-Instruct-DPO-lima-finetuned"

In [10]:
# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained(peft_model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [11]:
# LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'v_proj', 'q_proj', 'dense']
)

In [12]:
# Load the base model with BitsAndBytes configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [13]:
# Load the entire model on the GPU 0
device_map = {"": 0}

In [14]:
# Loads model from hugging face and device mapping
model = AutoModelForCausalLM.from_pretrained(
    peft_model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

model.config.use_cache = False

#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [16]:
# Training arguments
training_args = DPOConfig(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=50, # we set up the max_steps to 50, due to free GPU useage
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model_name,
    optim="paged_adamw_32bit",
    warmup_steps=5,
)

In [17]:
# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=dataset_modified,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=512,
    max_length=1024,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_prompt_length, max_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:389: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:402: UserWarning: You passed `max_prompt_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:442: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [18]:
gc.collect()
torch.cuda.empty_cache()

In [19]:
# Fine-tune model with DPO
dpo_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.693100
4,0.692300
5,0.694100
6,0.694900
7,0.639700
8,0.574700
9,0.567400
10,0.462800


TrainOutput(global_step=50, training_loss=0.2781807398796082, metrics={'train_runtime': 4127.4524, 'train_samples_per_second': 0.097, 'train_steps_per_second': 0.012, 'total_flos': 0.0, 'train_loss': 0.2781807398796082, 'epoch': 8.0})

In [21]:
# Save artifacts
dpo_trainer.model.save_pretrained("dpo_finetuned_model")
tokenizer.save_pretrained("dpo_finetuned_model")

('dpo_finetuned_model/tokenizer_config.json',
 'dpo_finetuned_model/special_tokens_map.json',
 'dpo_finetuned_model/tokenizer.model',
 'dpo_finetuned_model/added_tokens.json',
 'dpo_finetuned_model/tokenizer.json')

In [22]:
# Flush memory
del dpo_trainer, model
gc.collect()
torch.cuda.empty_cache()

In [23]:
# Reload model in FP16 (instead of NF4)
base_model = AutoModelForCausalLM.from_pretrained(
    peft_model_name,
    return_dict=True,
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
tokenizer = AutoTokenizer.from_pretrained(peft_model_name)

In [25]:
# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "dpo_finetuned_model")
model = model.merge_and_unload()

In [26]:
# Save model and tokenizer
model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)

('Mistral-7B-Instruct-DPO-lima-finetuned/tokenizer_config.json',
 'Mistral-7B-Instruct-DPO-lima-finetuned/special_tokens_map.json',
 'Mistral-7B-Instruct-DPO-lima-finetuned/tokenizer.model',
 'Mistral-7B-Instruct-DPO-lima-finetuned/added_tokens.json',
 'Mistral-7B-Instruct-DPO-lima-finetuned/tokenizer.json')

In [27]:
model.to("cuda")

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

#### Model Inference

In [28]:
import pandas as pd
df = pd.DataFrame(dataset_modified)

In [29]:
responses=[]
for i in tqdm(range(len(df))):
    instruction = df['prompt'].values[i]
    prompt = "[INST] " + instruction +  " [/INST]"
    inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs, max_new_tokens = 100, temperature=0.9, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    responses.append(response.split('[/INST] ')[-1])

  0%|          | 0/50 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
  2%|▏         | 1/50 [00:05<04:52,  5.97s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  4%|▍         | 2/50 [00:10<04:18,  5.39s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mas

In [30]:
df['model_response_output'] = responses

In [31]:
df.head()

,prompt,chosen,rejected,model_response_output
0,Can I spend the night alone in a tent in a for...,Living in a tent in a forest outside Stockholm...,Living in a tent in a forest outside Stockholm...,Camping in a forest outside Stockholm during e...
1,This claim was made popular by being said in t...,"Yes, that is a statement made in the movie ""Th...","Yes, that claim was made in the movie ""The Soc...",IQ (Intelligence Quotient) scores are a measur...
2,How do bats use sound to locate prey?,Bats use a biological sonar system called Echo...,Bats use a biological sonar system called echo...,Bats use a biological sonar system called Echo...
3,As a customer service representative. How woul...,"Firstly, I would apologize for the inconvenien...",I would handle an angry customer about a delay...,"I'm glad you asked me that question, as I beli..."
4,I want to write a software design document for...,**Overview**\n\nChateval is a software tool de...,title: Chateval: A Software Platform for Evalu...,Title: Chateval: A Versatile AI Evaluation Pla...


#### Push the Model to HuggingFace

In [32]:
model.push_to_hub(new_model_name, check_pr=True)
tokenizer.push_to_hub(new_model_name,check_pr=True)

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jkkn/Mistral-7B-Instruct-DPO-lima-finetuned/commit/fad3fa47c28d12048dc831e52028aca9547875c5', commit_message='Upload tokenizer', commit_description='', oid='fad3fa47c28d12048dc831e52028aca9547875c5', pr_url=None, pr_revision=None, pr_num=None)